In [ ]:
!git clone 'https://github.com/cambridgeltl/MTL-Bioinformatics-2016.git'

Cloning into 'MTL-Bioinformatics-2016'...
remote: Enumerating objects: 426, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 426 (delta 88), reused 86 (delta 86), pack-reused 319
Receiving objects: 100% (426/426), 25.55 MiB | 10.78 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (243/243), done.


In [ ]:
import pandas as pd

In [ ]:
c = 0

In [ ]:
def convert_format(file_name):
    global c
    f = open(file_name)
    data = f.read()
    sentences = data.split('\n\n')
    diseases = []
    tags = []
    sentence_id = []

    
    for sent in sentences:
        try:
            tokens = sent.split('\n')
            for token in tokens:
                # print(token)
                word,tag = token.split('\t')
                diseases.append(word)
                tags.append(tag)
                sentence_id.append(c)
            c+=1
        except:
            pass
    
    df = pd.DataFrame(columns=['words','labels','sentence_id'])
    df['words'] = diseases
    df['labels'] = tags
    df['sentence_id'] = sentence_id

    return df

In [ ]:
train_df1 = convert_format('/content/MTL-Bioinformatics-2016/data/BC5CDR-disease-IOB/train.tsv')
train_df1.head(10)

,words,labels,sentence_id
0,Selegiline,O,0
1,-,O,0
2,induced,O,0
3,postural,B-Disease,0
4,hypotension,I-Disease,0
5,in,O,0
6,Parkinson,B-Disease,0
7,',I-Disease,0
8,s,I-Disease,0
9,disease,I-Disease,0


In [ ]:
train_df1 = convert_format('/content/MTL-Bioinformatics-2016/data/BC5CDR-disease-IOB/train.tsv')
dev_df1 = convert_format('/content/MTL-Bioinformatics-2016/data/BC5CDR-disease-IOB/devel.tsv')
test_df1 = convert_format('/content/MTL-Bioinformatics-2016/data/BC5CDR-disease-IOB/test.tsv')

train_df2 = convert_format('/content/MTL-Bioinformatics-2016/data/NCBI-disease-IOB/train.tsv')
dev_df2 = convert_format('/content/MTL-Bioinformatics-2016/data/NCBI-disease-IOB/devel.tsv')
test_df2 = convert_format('/content/MTL-Bioinformatics-2016/data/NCBI-disease-IOB/test.tsv')

In [ ]:
train_df1.shape,dev_df1.shape,test_df1.shape

((118170, 3), (117453, 3), (124750, 3))

In [ ]:
train_df2.shape,dev_df2.shape,test_df2.shape

((135701, 3), (23969, 3), (24497, 3))

In [ ]:
complete_final_df = pd.concat([train_df1,dev_df1,test_df1,train_df2,dev_df2,test_df2])

In [ ]:
train_df3 = convert_format('/content/MTL-Bioinformatics-2016/data/BC4CHEMD/train.tsv')
dev_df3 = convert_format('/content/MTL-Bioinformatics-2016/data/BC4CHEMD/devel.tsv')
test_df3 = convert_format('/content/MTL-Bioinformatics-2016/data/BC4CHEMD/test.tsv')

train_df4 = convert_format('/content/MTL-Bioinformatics-2016/data/BC5CDR-IOB/train.tsv')
dev_df4 = convert_format('/content/MTL-Bioinformatics-2016/data/BC5CDR-IOB/devel.tsv')
test_df4 = convert_format('/content/MTL-Bioinformatics-2016/data/BC5CDR-IOB/test.tsv')

In [ ]:
train_df3['labels'].value_counts()

O             826726
I-Chemical     35744
B-Chemical     29478
Name: labels, dtype: int64

In [ ]:
train_df4['labels'].value_counts()

O             103967
B-Chemical      5203
B-Disease       4182
I-Disease       2918
I-Chemical      1900
Name: labels, dtype: int64

In [ ]:
train_df3_4 = pd.concat([train_df3,dev_df3,test_df3,train_df4,dev_df4,test_df4])

In [ ]:
from tqdm import tqdm

In [ ]:
sents_without_entities = []
for sent_id in tqdm(train_df3_4['sentence_id'].unique()):
    mini_df = train_df3_4[train_df3_4['sentence_id']==sent_id]
    labels = mini_df['labels'].to_list()
    if len(labels)==labels.count('O'):
        sents_without_entities.append(sent_id)

100%|██████████| 101623/101623 [09:10<00:00, 184.74it/s]


In [ ]:
len(sents_without_entities)

48530

In [ ]:
len(train_df3_4['sentence_id'].unique())

101623

In [ ]:
filtered_df = train_df3_4[~train_df3_4.sentence_id.isin(sents_without_entities)]

In [ ]:
filtered_df.head()

,words,labels,sentence_id
195,Chronic,O,25791
196,administration,O,25791
197,of,O,25791
198,haloperidol,B-Chemical,25791
199,increased,O,25791


In [ ]:
complete_final_df.shape,filtered_df.shape

((544540, 3), (1659732, 3))

In [ ]:
complete_final_df = pd.concat([complete_final_df,filtered_df])

In [ ]:
complete_final_df.shape

(2204272, 3)

In [ ]:
complete_final_df.isna().sum()

words          0
labels         0
sentence_id    0
dtype: int64

In [ ]:
complete_final_df.head()

,words,labels,sentence_id
0,Selegiline,O,4560
1,-,O,4560
2,induced,O,4560
3,postural,B-Disease,4560
4,hypotension,I-Disease,4560


In [ ]:
complete_final_df['labels'].unique()

array(['O', 'B-Disease', 'I-Disease', 'B-Chemical', 'I-Chemical'],
      dtype=object)

In [ ]:
all_ids = list(complete_final_df['sentence_id'].unique())

In [ ]:
total = len(all_ids)

In [ ]:
import random

In [ ]:

valid_count = int(total*0.1)
test_count = int(total*0.15)
train_count = total-valid_count-test_count

In [ ]:
test_stu_ids = random.sample(all_ids,test_count)

In [ ]:
valid_stu_ids = random.sample(set(all_ids)-set(test_stu_ids),valid_count)

In [ ]:
train_stu_ids = set(all_ids)-set(test_stu_ids)-set(valid_stu_ids)

In [ ]:
len(train_stu_ids),len(valid_stu_ids),len(test_stu_ids)

(55740, 7431, 11147)

In [ ]:
train_df = complete_final_df[complete_final_df.sentence_id.isin(train_stu_ids)]
valid_df = complete_final_df[complete_final_df.sentence_id.isin(valid_stu_ids)]
test_df = complete_final_df[complete_final_df.sentence_id.isin(test_stu_ids)]

In [ ]:
train_df.head(10)

,words,labels,sentence_id
0,Selegiline,O,4560
1,-,O,4560
2,induced,O,4560
3,postural,B-Disease,4560
4,hypotension,I-Disease,4560
5,in,O,4560
6,Parkinson,B-Disease,4560
7,',I-Disease,4560
8,s,I-Disease,4560
9,disease,I-Disease,4560


In [ ]:
valid_df.head(10)

,words,labels,sentence_id
323,Further,O,4572
324,studies,O,4572
325,on,O,4572
326,effects,O,4572
327,of,O,4572
328,irrigation,O,4572
329,solutions,O,4572
330,on,O,4572
331,rat,O,4572
332,bladders,O,4572


In [ ]:
test_df.head(10)

,words,labels,sentence_id
232,Orthostatic,B-Disease,4568
233,hypotension,I-Disease,4568
234,was,O,4568
235,ameliorated,O,4568
236,4,O,4568
237,days,O,4568
238,after,O,4568
239,withdrawal,O,4568
240,of,O,4568
241,selegiline,O,4568


In [ ]:
train_df.to_csv('/content/drive/MyDrive/Project/combined_train_data.csv')
test_df.to_csv('/content/drive/MyDrive/Project/combined_test_data.csv')
valid_df.to_csv('/content/drive/MyDrive/Project/combined_valid_data.csv')

In [ ]:
import pandas as pd

In [ ]:
old_df = pd.read_csv('/content/drive/MyDrive/Project/combined_train_data.csv')

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Project/gpt-labelled.xlsx')

In [ ]:
df.head()

,text,gpt-dis,gpt-symp,gpt-drug
0,A 23-year-old white female presents with comp...,['allergies'],[],[]
1,Consult for laparoscopic gastric bypass.,['laparoscopic gastric bypass'],[],[]
2,2-D M-Mode. Doppler.,[],[],[]
3,2-D Echocardiogram,[],[],[]
4,Morbid obesity. Laparoscopic antecolic anteg...,['Morbid obesity'],[],['Laparoscopic antecolic antegastric Roux-en-Y...


In [ ]:
df.shape

(2347, 4)

In [ ]:
df['gpt-symp'].value_counts()

[]                                                                                                            1295
['']                                                                                                            48
['chest pain']                                                                                                  17
['abdominal pain']                                                                                              11
['shortness of breath']                                                                                         10
                                                                                                              ... 
['pain', ' swelling', ' stiffness in affected finger']                                                           1
['nerve conduction study tests diagnostic of carpal tunnel syndrome']                                            1
['external carotid artery occlusion', ' back-bleeding', ' good internal carotid 

In [ ]:
df.head(30)

,text,gpt-dis,gpt-symp,gpt-drug
0,A 23-year-old white female presents with comp...,['allergies'],[],[]
1,Consult for laparoscopic gastric bypass.,['laparoscopic gastric bypass'],[],[]
2,2-D M-Mode. Doppler.,[],[],[]
3,2-D Echocardiogram,[],[],[]
4,Morbid obesity. Laparoscopic antecolic anteg...,['Morbid obesity'],[],['Laparoscopic antecolic antegastric Roux-en-Y...
5,"Liposuction of the supraumbilical abdomen, re...",[],[],[]
6,Suction-assisted lipectomy - lipodystrophy of...,['lipodystrophy'],"['abdominal pain', ' thigh pain']",[]
7,Echocardiogram and Doppler,['tudy are normal'],[''],[]
8,Morbid obesity. Laparoscopic Roux-en-Y gastr...,['Morbid obesity'],[],"['Laparoscopic Roux-en-Y gastric bypass', ' an..."
9,"Normal left ventricle, moderate biatrial enla...",[],[],[]


In [ ]:
from ast import literal_eval

In [ ]:
def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll-1

In [ ]:
find_sub_list(['bradycardia'],['Holter', 'monitoring', 'For', 'bradycardia', 'and', 'dizziness'])

(3, 3)

In [ ]:
import re

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
URL = 'https://en.wikipedia.org/wiki/List_of_medical_symptoms'
r = requests.get(URL)

soup = BeautifulSoup(r.content, 'html.parser')
# print(soup.prettify())

#print(soup.find_all(class_='_1RWRBu'))
find_obj = soup.find_all('div',attrs={'class':'div-col'})

In [ ]:
all_diseases = []

In [ ]:
for a in find_obj[1].find_all('a', href=True):
    all_diseases.append(a.text)

In [ ]:
len(all_diseases)

182

In [ ]:
all_diseases

['anorexia',
 'weight loss',
 'cachexia',
 'chills',
 'shivering',
 'convulsions',
 'deformity',
 'discharge',
 'dizziness',
 'Vertigo',
 'fatigue',
 'malaise',
 'asthenia',
 'hypothermia',
 'jaundice',
 'muscle weakness',
 'pyrexia',
 'sweats',
 'swelling',
 'swollen or painful lymph node(s)',
 'weight gain',
 'arrhythmia',
 'bradycardia',
 'chest pain',
 'claudication',
 'palpitations',
 'tachycardia',
 'dry mouth',
 'epistaxis',
 'halitosis',
 'hearing loss',
 'nasal discharge',
 'otalgia',
 'otorrhea',
 'sore throat',
 'toothache',
 'tinnitus',
 'trismus',
 'abdominal pain',
 'bloating',
 'belching',
 'bleeding',
 'Hematemesis',
 'blood in stool',
 'melena',
 'hematochezia',
 'constipation',
 'diarrhea',
 'dysphagia',
 'dyspepsia',
 'fecal incontinence',
 'flatulence',
 'heartburn',
 'nausea',
 'odynophagia',
 'proctalgia fugax',
 'pyrosis',
 'Rectal tenesmus',
 'steatorrhea',
 'vomiting',
 'alopecia',
 'hirsutism',
 'hypertrichosis',
 'Nail_disease §\xa0Nail_changes_and_conditions

In [ ]:
syntetic_data =[]

In [ ]:
import random

In [ ]:
def construct_sythetic_data(text,symp,replace_text_list):
    text = text.lower()
    text = re.sub(r'[^A-Za-z ]','',text)
    symp_list = literal_eval(symp)
    if len(symp_list)==0:
        return []
    symp = random.choice(symp_list)

    if symp not in text:
        return []
    if len(replace_text_list) == 2:
        replace_text = ' and '.join(replace_text_list)
    elif len(replace_text_list) == 3:
        replace_text = replace_text_list[0]+', '+replace_text_list[1]+' and '+replace_text_list[2]
    else:
        replace_text = replace_text_list[0]
    
    final_text = text.replace(symp,replace_text)
    return [final_text,[''],replace_text_list,['']]

In [ ]:
test_row = df.iloc[32].to_list()
test_row

[' Ultrasound examination of the scrotum due to scrotal pain.  Duplex and color flow imaging as well as real time gray-scale imaging of the scrotum and testicles was performed. ',
 '[]',
 "['scrotal pain']",
 '[]']

In [ ]:
construct_sythetic_data(test_row[0],test_row[2],['bodypain'])

[' ultrasound examination of the scrotum due to bodypain  duplex and color flow imaging as well as real time grayscale imaging of the scrotum and testicles was performed ',
 [''],
 ['bodypain'],
 ['']]

In [ ]:
number_of_synthteic_samples = 5000

In [ ]:
from tqdm import tqdm
import numpy as np

In [ ]:
temp_new_df = df[~df['gpt-symp'].isin(['[]','['']','[' ']'])]
temp_new_df.shape

(1052, 4)

In [ ]:
synthetic_data_frame = pd.DataFrame(columns=df.columns)

In [ ]:
c = 0
while(c<number_of_synthteic_samples):
    number_of_replacements = list(np.random.choice([1,2,3],1,p=[0.65,0.25,0.1]))[0]
    replace_symptom_list = random.sample(all_diseases,number_of_replacements)
    random_row = temp_new_df.iloc[random.choice(list(range(temp_new_df.shape[1])))].to_list()
    #print(random_row[0],random_row[2],replace_symptom_list)
    syntetic_row = construct_sythetic_data(random_row[0],random_row[2],replace_symptom_list)
    if syntetic_row==[]:
        continue
    synthetic_data_frame.loc[len(synthetic_data_frame)] = syntetic_row
    c += 1

    if c%100==0:
        print(c)

In [ ]:
synthetic_data_frame.head(60)

In [ ]:
df = pd.concat([df,synthetic_data_frame])

In [ ]:
def construct_BIO(text,symp):
    # print(text,symp)
    try:
        text = text.lower()
        text = re.sub(r'[^A-Za-z ]','',text)
        symp_list = literal_eval(symp)
        new_symp = []
        if len(symp) == 0:
            return [],[]
        for symp in symp_list:
            if text.count(symp)>2:
                return [],[]
            if symp.strip()=='':
                continue
            else:
                new_symp.append(symp.strip())
        
        if len(new_symp) == 0:
            return [],[]
        
        symp_ind = []
        words = text.split()
        for symp in new_symp:
            symp_words = symp.split()
            symp_ind.append(find_sub_list(symp_words,words))
        
        BIO_tag = ['O']*len(words)
        for inds in symp_ind:
            BIO_tag[inds[0]] = 'B-Symptom'
            for i in range(inds[0]+1,inds[1]+1):
                BIO_tag[i] = 'I-Symptom'

        return words,BIO_tag
    except:
        return [],[]

In [ ]:
construct_BIO(df['text'].to_list()[5200],df['gpt-symp'].to_list()[23])

([], [])

In [ ]:
sent_ids = []
tokens_list = []
tags_list = []

In [ ]:
from tqdm import tqdm

In [ ]:
old_id = list(old_df['sentence_id'].unique())

In [ ]:
import random

In [ ]:
c = 100000
for ind,(text,tags) in tqdm(enumerate(zip(df['text'].to_list(),df['gpt-symp'].to_list()))):
    
    tokens,tags = construct_BIO(text,tags)
    tokens_list += tokens
    tags_list += tags
    sent_ids += [ind+c]*len(tokens)

7347it [00:00, 102785.32it/s]


In [ ]:
len(sent_ids)

27516

In [ ]:
new_df = pd.DataFrame(columns = ['Unnamed: 0','words','labels','sentence_id'])

In [ ]:
new_df['Unnamed: 0']= [0]*len(tags_list)
new_df['words'] = tokens_list
new_df['labels'] = tags_list
new_df['sentence_id'] = sent_ids

In [ ]:
new_df.to_csv('/content/drive/MyDrive/Project/combined_symptom_train_data_tets_100000.csv',index=False)

In [ ]:
final_df = pd.concat([old_df,new_df])

In [ ]:
final_df = final_df.drop_duplicates()

In [ ]:
final_df.to_csv('/content/drive/MyDrive/Project/combined_train_data2_test_100000.csv',index=False)